# Schach Spiel mit importierem S_n

## Imports, die für dieses Notebook benötigt werden

In [7]:
import chess
from IPython.display import clear_output
from IPython.display import display
import json
import time
import random

## Ausgangssituation

Für dieses Board wurde die S_n Sequenz berechnet.

In [8]:
fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"
original_board = chess.Board(fen)

## Import der Daten

Die Daten wurden als FEN in der JSON-Datei Serialisiert.
Zum Initialisieren der Liste werden alle FENs gelesen und Board-Objekte erstellt.

In [9]:
S_n_sequence_new = []
f = open("S_n_seq.json", "r")
tmp = json.loads(f.read())
for item in tmp:
    tmp_list = []
    for board in item:
        tmp_list.append(chess.Board(board))
    S_n_sequence_new.append(tmp_list)
f.close()

## findBoardInSequence Hilfsfunktion

Diese Funktion durchsucht eine S_n Sequenz nach dem ersten Vorkommen eines übergebenen Board-Objekts.

Funktions-Argumente:
* situation: Das Board (als Objekt), welches gefunden werden soll
* sequence: Die S_n-Sequenz, in welcher das Board gesucht wird

Ergebnis der Ausführung:
* Die Funktion hat zwei mögliche Rückgaben:
  * Ein Tupel mit S_n Index (z.B. S_3) und Board-Index (z.B. 100).
    Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
  * Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Nebeneffekte:
Die Funktion verändert keinen der übergebenen Parameter.

Algorithmus:
1. Über die S_n-Sequenz iterieren.
2. Über jedes Board in einem spezifischem S_n iterieren.
3. Das Board mit dem _situation_ Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [10]:
def findBoardInSequence(situation, sequence):
    board_str = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        for j in range(len(sequence[i])):
            item = sequence[i][j]
            if board_str == (item.turn, item.__str__()):
                return i, j
    return -1,-1

## Finden des ersten Boards in der S_n-Sequenz

Mithilfe der zuvor definierten _findBoardInSequence_ Funktion wird das *original_board* in der *S_n_sequence* gesucht.

In [12]:
s_index = 0
board_index = 0

s_index, board_index = findBoardInSequence(original_board, S_n_sequence_new)

#Print search result
print("S" + str(s_index) + " - Board: " + str(board_index))

S9 - Board: 19108
Wall time: 4.24 s


## Berechnen der Züge bis zum Spielende
Diese Berechnung erfolgt mit zufälligen Zügen für den Spieler "Schwarz".

Voraussetzungen:
  * Die vorherige Zelle muss ausgeführt sein; Es müssen ein s_index und ein board_index gesetzt sein.

Ablauf:
  * Das aktuelle Board kopieren (Auswählen aus S_n und kopieren)
  * Überprüfen ob Weiß oder Schwarz am Zug ist:
    * Wenn Weiß:
      * Alle legal moves ausprobieren und überprüfen in welcher S_n Menge das Board sich hinterher befindet.
      * Den legal Move, mit dem geringsten n auswählen
    * Wenn Schwarz:
      * Einen zufälligen (legal-)Move durchführen
      * Überprüfen in welcher S_n Menge das Board sich nun befindet
  * s_index und board_index aktualisieren
  * Wiederholen bis, das Board in S_0 ist (s_index = 0) oder kein Move gefunden wurde (s_index = -1)

In [13]:
moves = []
while s_index > 0:
    curr_board = S_n_sequence_new[s_index][board_index].copy()
    if curr_board.turn:
        placement_dict = {}
        for move in curr_board.legal_moves:
            curr_board.push(move)
            s_index, board_index = findBoardInSequence(curr_board, S_n_sequence_new)
            if s_index != -1:
                placement_dict[(move,s_index,board_index)] = s_index
            curr_board.pop()
        move, s_index, board_index = min(placement_dict, key=placement_dict.get)
        print(move)
        moves.append(move)

    else:
        move_list = list(curr_board.legal_moves)
        move_index = random.randint(0, len(move_list) - 1)
        curr_board.push(move_list[move_index])
        s_index, board_index = findBoardInSequence(curr_board, S_n_sequence_new)
        if s_index == -1:
            print("No solution")
            break
        print(move_list[move_index])
        moves.append(move_list[move_index])


print(str(len(moves)) + " Züge bis zum Gewinn bestimmt")

KeyboardInterrupt: 

## Anzeigen des Ergebnisses
Ergebnisse anzeigen, in dem mit 2 Sekunden Verzögerung alle Moves durchgeführt werden.

In [ ]:
test_board = original_board.copy()
display(test_board)
time.sleep(2)
for move in moves:
    test_board.push(move)
    clear_output(wait=True)
    display(test_board)
    time.sleep(2)